# 说明：

（1）两种预处理方法：传统的标准化
- 标准化：数据减均值，除以方差 —— **均值为0，标准差为1**
- 归一化：减去最小值，除以(最大值-最小值) —— 所有数值范围落入[0,1]之间；例如图像/255（特例操作）

（2）为什么要做“**批**”标准化/归一化：Batch Normalization
- 因为虽然在训练前数据已经满足了标准化、归一化的条件；但是每一次训练，尤其是训练中“**每层**”都乘以偏置、每个图片都被进行卷积核池化处理；把“**第一层**”处理完的数据送入“**下一层**”时，数据就不再满足“标准化/归一化”了！—— **批标准化：每一层结束后，都在其后跟一个标准化处理**。—— **批标准化 = （各层进行）标准化 + 归一化**

（3）批标准化/归一化的作用/优势：
- 解决梯度消失 + 梯度爆炸；是一种训练优化方法。—— 激活函数(如sigmoid)两端，斜率变化都很慢！每次让数据回到[0,1]附近
- 防止过拟合、**提高泛化能力** —— 每一层输出都被重新标准化，都在同一范围内，故可接收各种数据，泛化能力增强！
- 加速训练 —— 允许使用更大的训练速率！因为每次都恢复到[0,1]附近，此时的激活函数变化(斜率)最大，即收敛速度最快！！
- 网络可以更深！—— 构建100层以上深网络：ResNet50、Inception等都有！ —— 残差网络也可以！

（4）实现的方法：在keras中也作为一“**层**”，一般称为“**BN层**”
- 函数：tf.keras.layers.Batchnormalization()

（5）批标准化处理的单位：每一个“**批次**”的数据；即每一个卷积层、全连接层后都要跟一个BN层（池化层后不用跟）

（6）批标准化的内部实现过程：对每一批次(batch)的数据
- 求每一个训练批次（每个batch里的那些数据）数据的“**均值**”；
- 求每一个训练批次数据的“**方差**”；
- 对每一个训练批次数据进行标准化：减均值，除以方差；
- 计算训练参数$\gamma$、$\beta$：就是自动记录“**批标准化前后**”数据间的差异 —— 就是一个线性变换

（7）使用了批标准化网络的训练、预测过程：
- 训练时：自动记录了**每个batch**的“**均值 + 方差**”
- 预测时：已经记录好了所有batch（整个训练集）的“均值 + 方程”；预测时计算“**整个训练样本**”的均值和方差（就是所有batch的“**均值的均值**” + “**方差的均值**”），来作为预测过程中BN层的均值和方差。

（8）批标准化层的参数：如(7)，批标准化有两种模式 —— 训练模式、预测模式 —— 哪种模式由training参数来定
- training = True：训练模式；该批标准化层将使用“**当前批输入**”的均值和方差，并对当前输入进行标准化
- trainging = False：预测模式；该批标准化层将使用“**整个训练集**”的均值和方差，并对当前输入进行标准化

（9）批标准化的使用位置：
- 原文：卷积层/全连接层 —— 批标准化层 —— 非线性激活函数层（也是一层）
- 实际：卷积层/全连接层 —— 非线性激活 —— 批标准化层函数层

# 批标准化的使用例子：修改航空图像分类

注：在文件4.1中，只用修改网络搭建即可

批标准化对每一个卷积层、全连接层都要处理；有两种搭建顺序均可：
- 卷积层 —— BN层 —— 激活函数；（原论文）
- 卷积层 —— 激活函数 —— BN层；（新推荐）

In [1]:
import tensorflow as tf

### 1. 原始网络搭建：

In [ ]:
model = tf.keras.Sequential()   #顺序模型

# 第一块：
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(128, 128, 3), activation='relu'))  # 卷积层、激活函数写在一起了
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

# 第二块：
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

# 第三块：
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

# 第四块：
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

# 第五块：
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

# 第六块：
model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'))

# 第七块：
model.add(tf.keras.layers.GlobalAveragePooling2D())

# 第八块：
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

### 2. 修改后网络搭建：选用方式1 —— 卷积层/全连接层、BN层、激活函数

In [ ]:
model = tf.keras.Sequential()   #顺序模型

# 第一块：
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(128, 128, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))    # 激活层

model.add(tf.keras.layers.Conv2D(64, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))    # 第二卷积层
          
model.add(tf.keras.layers.MaxPooling2D()) # 池化层：无需批标准化

# 第二块：
model.add(tf.keras.layers.Conv2D(128, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))    # 第一卷积层
          
model.add(tf.keras.layers.Conv2D(128, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))    # 第二卷积层
          
model.add(tf.keras.layers.MaxPooling2D()) # 池化层：无需批标准化

# 第三块：
model.add(tf.keras.layers.Conv2D(256, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))    # 第一卷积层
          
model.add(tf.keras.layers.Conv2D(256, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))    # 第二卷积层
          
model.add(tf.keras.layers.MaxPooling2D()) # 池化层：无需批标准化

# 第四块：
model.add(tf.keras.layers.Conv2D(512, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu')) 
          
model.add(tf.keras.layers.MaxPooling2D())

# 第五块：
model.add(tf.keras.layers.Conv2D(512, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
          
model.add(tf.keras.layers.MaxPooling2D())

# 第六块：
model.add(tf.keras.layers.Conv2D(1024, (3, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))  

# 第七块：全局池化，无需批标准化
model.add(tf.keras.layers.GlobalAveragePooling2D())

# 第八块：
model.add(tf.keras.layers.Dense(1024))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))   # 第一全连接层
          
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))   # 第二圈连接层
                    
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 最后的输出层，就不必了